In [1]:
import numpy as np
import tensorflow as tf

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load in data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../Tensorflow-applications/MNIST_data//", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../Tensorflow-applications/MNIST_data//train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../Tensorflow-applications/MNIST_data//train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../Tensorflow-applications/MNIST_data//t10k-images-idx3-ubyte.gz
Extracting ../Tensorflow-applications/MNIST_data//t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Settings

In [3]:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args

In [4]:
dim_learning_rate = Real(low=1e-4, high=0.1, prior='log-uniform', name='learning_rate')
dim_filt_cv1 = Integer(6,15, name='filt_cv1')
dim_filt_primaryCaps = Integer(6,15, name='filt_primaryCaps')
dim_chan_conv1 = Categorical([16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256], name='chan_conv1')
dim_chan_primaryCaps = Categorical([16,32,48,64], name='chan_primaryCaps')
dim_caps1_size = Integer(4,16, name='caps1_size')
#dim_caps2_size = Integer(10, name='caps2_size')
dim_pred_matrix_size = Integer(4,20, name='pred_matrix_size')
dim_routing_rounds = Integer(1,5, name='routing_rounds')
dim_batch = Categorical([32,64,96,128], name='batch')

In [5]:
dimensions = [dim_caps1_size,
             dim_pred_matrix_size,
             dim_chan_conv1,
             dim_filt_cv1,
             dim_chan_primaryCaps,
             dim_filt_primaryCaps,
             dim_routing_rounds,
              dim_learning_rate,
             dim_batch]

In [6]:
default_parameters = [8,16,256,9,32,9,3,0.001,64]

In [7]:
epsilon = 1e-7

## Squash function

In [8]:
def squash(input_vector, axis):
    normalised_input = tf.reduce_sum(tf.square(input_vector), axis = axis, keepdims = True)
    scale = tf.divide(normalised_input, tf.add(normalised_input, 1.))
    vector = tf.divide(input_vector, tf.sqrt(tf.add(normalised_input, epsilon)))
    output = tf.multiply(scale, vector)
    
    return(output)

## Convolutional layer

In [9]:
def convolutional(input_data, conv_shape, stride_shape, relu=True):
    weights = tf.Variable(tf.truncated_normal(conv_shape, stddev=0.3), name='W')
    bias = tf.Variable(tf.truncated_normal([conv_shape[-1]], stddev=0.3), name='B')
    out_layer = tf.nn.conv2d(input_data, weights, stride_shape, padding = 'VALID')
    out_layer_bias = tf.add(out_layer, bias)
    
    if relu == True:
        out_layer_final = tf.nn.relu(out_layer_bias)
        return(out_layer_final)
    
    return(out_layer_bias)

## Primarycaps

In [10]:
def primarycaps(input_data, conv_shape, stride_shape, primaryCaps_channels, caps1_size, caps2_size, pose_size, batch):
    output = convolutional(input_data, conv_shape, stride_shape, relu=False)
    filter_size = output.get_shape().as_list()[1]
    caps1_raw = tf.reshape(output, [-1,filter_size*filter_size*primaryCaps_channels,caps1_size], name='caps1_raw')
    caps1_output = squash(caps1_raw, axis=-1)
    caps1_output_expand = tf.expand_dims(caps1_output, axis=-1)
    caps1_output_expand2 = tf.expand_dims(caps1_output_expand, axis=2)
    caps1_output_expand2_tiled = tf.tile(caps1_output_expand2, [1,1,caps2_size,1,1], name = 'caps1_out_tiled')
    
    weight_matrix = tf.Variable(tf.truncated_normal([1, filter_size*filter_size*primaryCaps_channels, caps2_size, pose_size, caps1_size], stddev=0.1), name='W_matrix')
    weight_matrix_tiled = tf.tile(weight_matrix, [batch, 1, 1, 1, 1], name = 'W_matrix_tiled')
    caps2_predicted = tf.matmul(weight_matrix_tiled, caps1_output_expand2_tiled, name='caps2_predicted')
    
    #caps2_predicted = tf.einsum('abdc,iabdf->iabdf', weight_matrix, caps1_output_expand2_tiled)
    
    return(caps2_predicted)

## Routing by agreement

In [11]:
def routing_by_agreement(input_data, caps2_size, rounds, batch):
    raw_weights = tf.zeros([batch, input_data.get_shape().as_list()[1], caps2_size, 1, 1], name = 'raw_weights')
    
    for i in range(rounds):
        routing_weights = tf.nn.softmax(raw_weights, axis=2, name = 'routing_weights' + str(i))
        weighted_predictions = tf.multiply(routing_weights, input_data, name = 'weighted_predictions' + str(i))
        weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, name = 'weighted_sum' + str(i), keepdims = True)
        caps2_output = squash(weighted_sum, axis=-2)
    
        caps2_output_tiled = tf.tile(caps2_output, [1, input_data.get_shape().as_list()[1], 1, 1, 1], name = 'caps2_output_tiled'+ str(i))
        agreement = tf.matmul(input_data, caps2_output_tiled, transpose_a = True, name = 'agreement'+ str(i))
        raw_weights = tf.add(raw_weights, agreement, name = 'raw_weights' + str(i))
        
        #agreement = tf.einsum('', input_data, caps2_output)
        
    return(caps2_output)

## Graph

In [8]:
X = tf.placeholder(tf.float32, [None,784])
X_reshape = tf.reshape(X, shape = [-1,28,28,1])
y = tf.placeholder(tf.float32, [None,10])

In [9]:
conv1 = convolutional(X_reshape, [conv1_filter,conv1_filter,X_reshape.get_shape().as_list()[-1],
                                  conv1_channels],[1,1,1,1])
primary = primarycaps(conv1, [primaryCaps_filter,primaryCaps_filter,conv1.get_shape().as_list()[-1],
                              primaryCaps_channels*caps1_size], [1,2,2,1],primaryCaps_channels,
                      caps1_size,caps2_size, pred_matrix_size, batch=tf.shape(X)[0])
output = routing_by_agreement(primary, caps2_size, routing_rounds, batch=tf.shape(X)[0])

In [10]:
output

<tf.Tensor 'Mul_3:0' shape=(?, 1, 10, 16, 1) dtype=float32>

## DigitCaps

In [11]:
squared_norm_caps2 = tf.reduce_sum(tf.square(output), axis=-2, keepdims = True, name='caps2_norm')
caps2_activation = tf.sqrt(tf.add(squared_norm_caps2, epsilon), name = 'caps2_activation')

y_estimate = tf.squeeze(caps2_activation, axis=[1,3,4], name = 'y_pred')
y_pred = tf.nn.softmax(y_estimate, axis=1)

## Loss function

In [12]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_estimate, labels=y))
optimiser = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

## Make prediction

In [13]:
correct_prediction = tf.equal(tf.argmax(y,axis=1), tf.argmax(y_pred, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))

## Make function to generate graph

In [12]:
def create_graph(caps1_size, caps2_size,pred_matrix_size,conv1_channels,conv1_filter,primaryCaps_channels,primaryCaps_filter,routing_rounds,learning_rate, X_reshape, y, X):
    
    conv1 = convolutional(X_reshape, [conv1_filter,conv1_filter,X_reshape.get_shape().as_list()[-1],
                                  conv1_channels],[1,1,1,1])
    primary = primarycaps(conv1, [primaryCaps_filter,primaryCaps_filter,conv1.get_shape().as_list()[-1],
                              primaryCaps_channels*caps1_size], [1,2,2,1],primaryCaps_channels,
                      caps1_size,caps2_size, pred_matrix_size, batch=tf.shape(X)[0])
    output = routing_by_agreement(primary, caps2_size, routing_rounds, batch=tf.shape(X)[0])
    
    squared_norm_caps2 = tf.reduce_sum(tf.square(output), axis=-2, keepdims = True, name='caps2_norm')
    caps2_activation = tf.sqrt(tf.add(squared_norm_caps2, epsilon), name = 'caps2_activation')

    y_estimate = tf.squeeze(caps2_activation, axis=[1,3,4], name = 'y_pred')
    y_pred = tf.nn.softmax(y_estimate, axis=1)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = y_estimate, labels=y))
    optimiser = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)
    
    correct_prediction = tf.equal(tf.argmax(y,axis=1), tf.argmax(y_pred, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))
    
    return(optimiser, cross_entropy, accuracy)

## Setup bayesian hyperparameter search

In [13]:
@use_named_args(dimensions=dimensions)
def fitness(caps1_size,pred_matrix_size,chan_conv1,filt_cv1,chan_primaryCaps,filt_primaryCaps,routing_rounds,learning_rate,batch):

    epochs = 2

    #If run on AWS
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.allow_growth=True

    X = tf.placeholder(tf.float32, [None,784])
    X_reshape = tf.reshape(X, shape = [-1,28,28,1])
    y = tf.placeholder(tf.float32, [None,10])
    
    caps2_size = 10
    optimiser, cross_entropy, accuracy = create_graph(caps1_size,caps2_size,
             pred_matrix_size,
             chan_conv1,
             filt_cv1,
             chan_primaryCaps,
             filt_primaryCaps,
             routing_rounds,
              learning_rate, X_reshape, y, X)
    
    init_op = tf.global_variables_initializer()
    batch_size = batch
    acc_val = 0
    
    with tf.Session(config=config) as sess:
        init_op.run()

        total_batch = int(len(mnist.train.images)/batch_size)
        total_batch_validation = int(len(mnist.validation.images)/batch_size)

        for epoch in range(epochs):
            avg_cost = 0
            for i in range(total_batch):
                batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
                a, c= sess.run([optimiser, cross_entropy], feed_dict={X: batch_x, y: batch_y})
                avg_cost += c/total_batch
                print('batch:', i)
            print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
        
            acc_vals = []
            for iterations in range(total_batch_validation):
                batch_x, batch_y = mnist.validation.next_batch(batch_size=batch_size)
                val_acc = sess.run(accuracy, feed_dict={X: batch_x, y: batch_y})
                acc_vals.append(val_acc)
            acc_val = np.mean(acc_vals)
            print('val_acc:', acc_val)
            
    return(-acc_val)

In [ ]:
search_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            acq_func='EI', # Expected Improvement.
                            n_calls=11,
                            x0=default_parameters)

batch: 0
batch: 1
batch: 2
batch: 3
batch: 4
batch: 5
batch: 6
batch: 7
batch: 8
batch: 9
batch: 10
batch: 11
batch: 12
batch: 13
batch: 14
batch: 15
batch: 16
batch: 17
batch: 18
batch: 19
batch: 20
batch: 21
batch: 22
batch: 23
batch: 24
batch: 25
batch: 26
batch: 27
batch: 28
batch: 29
batch: 30
batch: 31
batch: 32
batch: 33
batch: 34
batch: 35
batch: 36
batch: 37
batch: 38
batch: 39
batch: 40
batch: 41
batch: 42
batch: 43
batch: 44
batch: 45
batch: 46
batch: 47
batch: 48
batch: 49
batch: 50
batch: 51
batch: 52
batch: 53
batch: 54
batch: 55
batch: 56
batch: 57
batch: 58
batch: 59
batch: 60
batch: 61
batch: 62
batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98
batch: 99
batch: 100

batch: 755
batch: 756
batch: 757
batch: 758
batch: 759
batch: 760
batch: 761
batch: 762
batch: 763
batch: 764
batch: 765
batch: 766
batch: 767
batch: 768
batch: 769
batch: 770
batch: 771
batch: 772
batch: 773
batch: 774
batch: 775
batch: 776
batch: 777
batch: 778
batch: 779
batch: 780
batch: 781
batch: 782
batch: 783
batch: 784
batch: 785
batch: 786
batch: 787
batch: 788
batch: 789
batch: 790
batch: 791
batch: 792
batch: 793
batch: 794
batch: 795
batch: 796
batch: 797
batch: 798
batch: 799
batch: 800
batch: 801
batch: 802
batch: 803
batch: 804
batch: 805
batch: 806
batch: 807
batch: 808
batch: 809
batch: 810
batch: 811
batch: 812
batch: 813
batch: 814
batch: 815
batch: 816
batch: 817
batch: 818
batch: 819
batch: 820
batch: 821
batch: 822
batch: 823
batch: 824
batch: 825
batch: 826
batch: 827
batch: 828
batch: 829
batch: 830
batch: 831
batch: 832
batch: 833
batch: 834
batch: 835
batch: 836
batch: 837
batch: 838
batch: 839
batch: 840
batch: 841
batch: 842
batch: 843
batch: 844
batch: 845

batch: 647
batch: 648
batch: 649
batch: 650
batch: 651
batch: 652
batch: 653
batch: 654
batch: 655
batch: 656
batch: 657
batch: 658
batch: 659
batch: 660
batch: 661
batch: 662
batch: 663
batch: 664
batch: 665
batch: 666
batch: 667
batch: 668
batch: 669
batch: 670
batch: 671
batch: 672
batch: 673
batch: 674
batch: 675
batch: 676
batch: 677
batch: 678
batch: 679
batch: 680
batch: 681
batch: 682
batch: 683
batch: 684
batch: 685
batch: 686
batch: 687
batch: 688
batch: 689
batch: 690
batch: 691
batch: 692
batch: 693
batch: 694
batch: 695
batch: 696
batch: 697
batch: 698
batch: 699
batch: 700
batch: 701
batch: 702
batch: 703
batch: 704
batch: 705
batch: 706
batch: 707
batch: 708
batch: 709
batch: 710
batch: 711
batch: 712
batch: 713
batch: 714
batch: 715
batch: 716
batch: 717
batch: 718
batch: 719
batch: 720
batch: 721
batch: 722
batch: 723
batch: 724
batch: 725
batch: 726
batch: 727
batch: 728
batch: 729
batch: 730
batch: 731
batch: 732
batch: 733
batch: 734
batch: 735
batch: 736
batch: 737

batch: 540
batch: 541
batch: 542
batch: 543
batch: 544
batch: 545
batch: 546
batch: 547
batch: 548
batch: 549
batch: 550
batch: 551
batch: 552
batch: 553
batch: 554
batch: 555
batch: 556
batch: 557
batch: 558
batch: 559
batch: 560
batch: 561
batch: 562
batch: 563
batch: 564
batch: 565
batch: 566
batch: 567
batch: 568
batch: 569
batch: 570
batch: 571
batch: 572
batch: 573
batch: 574
batch: 575
batch: 576
batch: 577
batch: 578
batch: 579
batch: 580
batch: 581
batch: 582
batch: 583
batch: 584
batch: 585
batch: 586
batch: 587
batch: 588
batch: 589
batch: 590
batch: 591
batch: 592
batch: 593
batch: 594
batch: 595
batch: 596
batch: 597
batch: 598
batch: 599
batch: 600
batch: 601
batch: 602
batch: 603
batch: 604
batch: 605
batch: 606
batch: 607
batch: 608
batch: 609
batch: 610
batch: 611
batch: 612
batch: 613
batch: 614
batch: 615
batch: 616
batch: 617
batch: 618
batch: 619
batch: 620
batch: 621
batch: 622
batch: 623
batch: 624
batch: 625
batch: 626
batch: 627
batch: 628
batch: 629
batch: 630

batch: 432
batch: 433
batch: 434
batch: 435
batch: 436
batch: 437
batch: 438
batch: 439
batch: 440
batch: 441
batch: 442
batch: 443
batch: 444
batch: 445
batch: 446
batch: 447
batch: 448
batch: 449
batch: 450
batch: 451
batch: 452
batch: 453
batch: 454
batch: 455
batch: 456
batch: 457
batch: 458
batch: 459
batch: 460
batch: 461
batch: 462
batch: 463
batch: 464
batch: 465
batch: 466
batch: 467
batch: 468
batch: 469
batch: 470
batch: 471
batch: 472
batch: 473
batch: 474
batch: 475
batch: 476
batch: 477
batch: 478
batch: 479
batch: 480
batch: 481
batch: 482
batch: 483
batch: 484
batch: 485
batch: 486
batch: 487
batch: 488
batch: 489
batch: 490
batch: 491
batch: 492
batch: 493
batch: 494
batch: 495
batch: 496
batch: 497
batch: 498
batch: 499
batch: 500
batch: 501
batch: 502
batch: 503
batch: 504
batch: 505
batch: 506
batch: 507
batch: 508
batch: 509
batch: 510
batch: 511
batch: 512
batch: 513
batch: 514
batch: 515
batch: 516
batch: 517
batch: 518
batch: 519
batch: 520
batch: 521
batch: 522

...

batch: 353
batch: 354
batch: 355
batch: 356
batch: 357
batch: 358
batch: 359
batch: 360
batch: 361
batch: 362
batch: 363
batch: 364
batch: 365
batch: 366
batch: 367
batch: 368
batch: 369
batch: 370
batch: 371
batch: 372
batch: 373
batch: 374
batch: 375
batch: 376
batch: 377
batch: 378
batch: 379
batch: 380
batch: 381
batch: 382
batch: 383
batch: 384
batch: 385
batch: 386
batch: 387
batch: 388
batch: 389
batch: 390
batch: 391
batch: 392
batch: 393
batch: 394
batch: 395
batch: 396
batch: 397
batch: 398
batch: 399
batch: 400
batch: 401
batch: 402
batch: 403
batch: 404
batch: 405
batch: 406
batch: 407
batch: 408
batch: 409
batch: 410
batch: 411
batch: 412
batch: 413
batch: 414
batch: 415
batch: 416
batch: 417
batch: 418
batch: 419
batch: 420
batch: 421
batch: 422
batch: 423
batch: 424
batch: 425
batch: 426
batch: 427
batch: 428
batch: 429
batch: 430
batch: 431
batch: 432
batch: 433
batch: 434
batch: 435
batch: 436
batch: 437
batch: 438
batch: 439
batch: 440
batch: 441
batch: 442
batch: 443

batch: 1090
batch: 1091
batch: 1092
batch: 1093
batch: 1094
batch: 1095
batch: 1096
batch: 1097
batch: 1098
batch: 1099
batch: 1100
batch: 1101
batch: 1102
batch: 1103
batch: 1104
batch: 1105
batch: 1106
batch: 1107
batch: 1108
batch: 1109
batch: 1110
batch: 1111
batch: 1112
batch: 1113
batch: 1114
batch: 1115
batch: 1116
batch: 1117
batch: 1118
batch: 1119
batch: 1120
batch: 1121
batch: 1122
batch: 1123
batch: 1124
batch: 1125
batch: 1126
batch: 1127
batch: 1128
batch: 1129
batch: 1130
batch: 1131
batch: 1132
batch: 1133
batch: 1134
batch: 1135
batch: 1136
batch: 1137
batch: 1138
batch: 1139
batch: 1140
batch: 1141
batch: 1142
batch: 1143
batch: 1144
batch: 1145
batch: 1146
batch: 1147
batch: 1148
batch: 1149
batch: 1150
batch: 1151
batch: 1152
batch: 1153
batch: 1154
batch: 1155
batch: 1156
batch: 1157
batch: 1158
batch: 1159
batch: 1160
batch: 1161
batch: 1162
batch: 1163
batch: 1164
batch: 1165
batch: 1166
batch: 1167
batch: 1168
batch: 1169
batch: 1170
batch: 1171
batch: 1172
batc

batch: 63
batch: 64
batch: 65
batch: 66
batch: 67
batch: 68
batch: 69
batch: 70
batch: 71
batch: 72
batch: 73
batch: 74
batch: 75
batch: 76
batch: 77
batch: 78
batch: 79
batch: 80
batch: 81
batch: 82
batch: 83
batch: 84
batch: 85
batch: 86
batch: 87
batch: 88
batch: 89
batch: 90
batch: 91
batch: 92
batch: 93
batch: 94
batch: 95
batch: 96
batch: 97
batch: 98
batch: 99
batch: 100
batch: 101
batch: 102
batch: 103
batch: 104
batch: 105
batch: 106
batch: 107
batch: 108
batch: 109
batch: 110
batch: 111
batch: 112
batch: 113
batch: 114
batch: 115
batch: 116
batch: 117
batch: 118
batch: 119
batch: 120
batch: 121
batch: 122
batch: 123
batch: 124
batch: 125
batch: 126
batch: 127
batch: 128
batch: 129
batch: 130
batch: 131
batch: 132
batch: 133
batch: 134
batch: 135
batch: 136
batch: 137
batch: 138
batch: 139
batch: 140
batch: 141
batch: 142
batch: 143
batch: 144
batch: 145
batch: 146
batch: 147
batch: 148
batch: 149
batch: 150
batch: 151
batch: 152
batch: 153
batch: 154
batch: 155
batch: 156
bat

batch: 812
batch: 813
batch: 814
batch: 815
batch: 816
batch: 817
batch: 818
batch: 819
batch: 820
batch: 821
batch: 822
batch: 823
batch: 824
batch: 825
batch: 826
batch: 827
batch: 828
batch: 829
batch: 830
batch: 831
batch: 832
batch: 833
batch: 834
batch: 835
batch: 836
batch: 837
batch: 838
batch: 839
batch: 840
batch: 841
batch: 842
batch: 843
batch: 844
batch: 845
batch: 846
batch: 847
batch: 848
batch: 849
batch: 850
batch: 851
batch: 852
batch: 853
batch: 854
batch: 855
batch: 856
batch: 857
batch: 858
batch: 859
batch: 860
batch: 861
batch: 862
batch: 863
batch: 864
batch: 865
batch: 866
batch: 867
batch: 868
batch: 869
batch: 870
batch: 871
batch: 872
batch: 873
batch: 874
batch: 875
batch: 876
batch: 877
batch: 878
batch: 879
batch: 880
batch: 881
batch: 882
batch: 883
batch: 884
batch: 885
batch: 886
batch: 887
batch: 888
batch: 889
batch: 890
batch: 891
batch: 892
batch: 893
batch: 894
batch: 895
batch: 896
batch: 897
batch: 898
batch: 899
batch: 900
batch: 901
batch: 902

batch: 1511
batch: 1512
batch: 1513
batch: 1514
batch: 1515
batch: 1516
batch: 1517
batch: 1518
batch: 1519
batch: 1520
batch: 1521
batch: 1522
batch: 1523
batch: 1524
batch: 1525
batch: 1526
batch: 1527
batch: 1528
batch: 1529
batch: 1530
batch: 1531
batch: 1532
batch: 1533
batch: 1534
batch: 1535
batch: 1536
batch: 1537
batch: 1538
batch: 1539
batch: 1540
batch: 1541
batch: 1542
batch: 1543
batch: 1544
batch: 1545
batch: 1546
batch: 1547
batch: 1548
batch: 1549
batch: 1550
batch: 1551
batch: 1552
batch: 1553
batch: 1554
batch: 1555
batch: 1556
batch: 1557
batch: 1558
batch: 1559
batch: 1560
batch: 1561
batch: 1562
batch: 1563
batch: 1564
batch: 1565
batch: 1566
batch: 1567
batch: 1568
batch: 1569
batch: 1570
batch: 1571
batch: 1572
batch: 1573
batch: 1574
batch: 1575
batch: 1576
batch: 1577
batch: 1578
batch: 1579
batch: 1580
batch: 1581
batch: 1582
batch: 1583
batch: 1584
batch: 1585
batch: 1586
batch: 1587
batch: 1588
batch: 1589
batch: 1590
batch: 1591
batch: 1592
batch: 1593
batc